### Домашнее задание №4

In [1]:
import urllib
import html
import pandas as pd
import re
import requests
import numpy as np
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
cyr_authors = ['Фрай М.', 'Хантер Э.', 'Емец Д.']
authors = [urllib.parse.quote(author, encoding='windows-1251') for author in cyr_authors]
authors

['%D4%F0%E0%E9%20%CC.', '%D5%E0%ED%F2%E5%F0%20%DD.', '%C5%EC%E5%F6%20%C4.']

In [3]:
url_format = 'https://www.chitai-gorod.ru/search/result.php?q={author}&type=author'

In [4]:
def get_book_url_from_div(div):
    return div.find_element_by_class_name('product-card__link').get_attribute('href')

def get_books_urls(driver, url, t_sleep=5):
    driver.get(url)
    books_count = int(driver.find_element_by_class_name("count-result__value").text)
    print("Предполагается книг: ", books_count)
    found_books = []
    while len(found_books) < books_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        sleep(t_sleep)
        show_more = driver.find_element_by_class_name("catalog__btn_show-more")
        if show_more:  
            wait = WebDriverWait(driver, t_sleep*2)
            wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn_normal')))
            click_script = "document.getElementsByClassName('catalog__btn_show-more')[0].\
                firstElementChild.click()"
            driver.execute_script(click_script)
            sleep(t_sleep)
        found_books = list(driver.find_elements_by_class_name("product-card"))
    print("Найдено книг:", len(found_books))
    urls = [get_book_url_from_div(div) for div in found_books]
    return urls

books_urls = []
with webdriver.Firefox() as driver:
    for a_id, _ in enumerate(authors):
        print(cyr_authors[a_id])
        url_params = {'author': authors[a_id]}
        url_current = url_format.format(**url_params)
        books_urls.extend(get_books_urls(driver, url_current))
    print("Найдено ссылок: ", len(books_urls))

Фрай М.
Предполагается книг:  153
Найдено книг: 153
Хантер Э.
Предполагается книг:  97
Найдено книг: 97
Емец Д.
Предполагается книг:  192
Найдено книг: 192
Найдено ссылок:  442


In [5]:
books_urls

['https://www.chitai-gorod.ru/catalog/book/1085970/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1068930/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1059170/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1051063/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1021715/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1017577/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1012908/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1009680/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1008621/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1007486/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/1004167/?watch_fromlist=search_result',
 'https://www.chitai-gorod.ru/catalog/book/996307/?watch_fromlist=search_result',
 'htt

In [6]:
def get_id(soup):
    price = soup.find('div', class_='js-analytic-product-page').attrs['data-product']
    return {'ID карточки': int(price)}

def get_header(soup):
    result = {}
    header = soup.find('div', class_='product__header')
    result['Название'] = header.find('h1', class_='js-analytic-product-title').text
    result['Автор'] = header.find('a', class_='product__author').text.strip()
    return result

def get_rating(soup):
    result = {}
    rating = soup.find('div', class_='rating')
    span = rating.find('span').text
    result['Рейтинг'] = int(re.findall('^\d+', span)[0])
    result['Голоса'] = re.findall('\(\d+\)', span)[0]
    result['Голоса'] = int(re.sub('[\(\)]', '', result['Голоса']))
    return result

def get_price(soup):
    price_div = soup.find('div', class_='product__price')
    if price_div:
        price = price_div.find('div', class_='price').text
        price = re.findall('\d+', price)[0]
        return {'Цена:': int(price)}
    return {}

def get_cover(soup):
    cover = soup.find('div', class_='product__image')
    link = cover.find('img').attrs['src']
    return {'Обложка': link}

def get_more_properties(soup):
    result = {}
    props = soup.find('div', class_='product__props')
    props = props.find_all('div', class_='product-prop')
    for prop in props:
        title = prop.find('div', class_='product-prop__title').text.strip()
        value = prop.find('div', class_='product-prop__value').text.strip()
        if re.match('^\d+$', value):
            value = int(value)
        result[title] = value
    return result

def extract_book_info(soup):
    book_info = {}
    book_info.update(get_id(soup))
    book_info.update(get_header(soup))
    book_info.update(get_rating(soup))
    book_info.update(get_price(soup))
    book_info.update(get_more_properties(soup))
    book_info.update(get_cover(soup))
    return book_info

result = []
for url_id, url in enumerate(books_urls):
    print(url_id, url)
    book_html = requests.get(url).text
    soup = BeautifulSoup(book_html, 'html.parser')
    result.append(extract_book_info(soup))

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)
df.shape
df.head(10)

0 https://www.chitai-gorod.ru/catalog/book/1085970/?watch_fromlist=search_result
1 https://www.chitai-gorod.ru/catalog/book/1068930/?watch_fromlist=search_result
2 https://www.chitai-gorod.ru/catalog/book/1059170/?watch_fromlist=search_result
3 https://www.chitai-gorod.ru/catalog/book/1051063/?watch_fromlist=search_result
4 https://www.chitai-gorod.ru/catalog/book/1021715/?watch_fromlist=search_result
5 https://www.chitai-gorod.ru/catalog/book/1017577/?watch_fromlist=search_result
6 https://www.chitai-gorod.ru/catalog/book/1012908/?watch_fromlist=search_result
7 https://www.chitai-gorod.ru/catalog/book/1009680/?watch_fromlist=search_result
8 https://www.chitai-gorod.ru/catalog/book/1008621/?watch_fromlist=search_result
9 https://www.chitai-gorod.ru/catalog/book/1007486/?watch_fromlist=search_result
10 https://www.chitai-gorod.ru/catalog/book/1004167/?watch_fromlist=search_result
11 https://www.chitai-gorod.ru/catalog/book/996307/?watch_fromlist=search_result
12 https://www.chitai-gorod

102 https://www.chitai-gorod.ru/catalog/book/361307/?watch_fromlist=search_result
103 https://www.chitai-gorod.ru/catalog/book/358727/?watch_fromlist=search_result
104 https://www.chitai-gorod.ru/catalog/book/358088/?watch_fromlist=search_result
105 https://www.chitai-gorod.ru/catalog/book/354742/?watch_fromlist=search_result
106 https://www.chitai-gorod.ru/catalog/book/354139/?watch_fromlist=search_result
107 https://www.chitai-gorod.ru/catalog/book/353710/?watch_fromlist=search_result
108 https://www.chitai-gorod.ru/catalog/book/350170/?watch_fromlist=search_result
109 https://www.chitai-gorod.ru/catalog/book/324455/?watch_fromlist=search_result
110 https://www.chitai-gorod.ru/catalog/book/323381/?watch_fromlist=search_result
111 https://www.chitai-gorod.ru/catalog/book/312706/?watch_fromlist=search_result
112 https://www.chitai-gorod.ru/catalog/book/310896/?watch_fromlist=search_result
113 https://www.chitai-gorod.ru/catalog/book/310895/?watch_fromlist=search_result
114 https://www.

202 https://www.chitai-gorod.ru/catalog/book/537992/?watch_fromlist=search_result
203 https://www.chitai-gorod.ru/catalog/book/454268/?watch_fromlist=search_result
204 https://www.chitai-gorod.ru/catalog/book/444313/?watch_fromlist=search_result
205 https://www.chitai-gorod.ru/catalog/book/442954/?watch_fromlist=search_result
206 https://www.chitai-gorod.ru/catalog/book/440184/?watch_fromlist=search_result
207 https://www.chitai-gorod.ru/catalog/book/439197/?watch_fromlist=search_result
208 https://www.chitai-gorod.ru/catalog/book/437293/?watch_fromlist=search_result
209 https://www.chitai-gorod.ru/catalog/book/417841/?watch_fromlist=search_result
210 https://www.chitai-gorod.ru/catalog/book/398475/?watch_fromlist=search_result
211 https://www.chitai-gorod.ru/catalog/book/380218/?watch_fromlist=search_result
212 https://www.chitai-gorod.ru/catalog/book/380215/?watch_fromlist=search_result
213 https://www.chitai-gorod.ru/catalog/book/379997/?watch_fromlist=search_result
214 https://www.

302 https://www.chitai-gorod.ru/catalog/book/793991/?watch_fromlist=search_result
303 https://www.chitai-gorod.ru/catalog/book/782786/?watch_fromlist=search_result
304 https://www.chitai-gorod.ru/catalog/book/776541/?watch_fromlist=search_result
305 https://www.chitai-gorod.ru/catalog/book/770141/?watch_fromlist=search_result
306 https://www.chitai-gorod.ru/catalog/book/768211/?watch_fromlist=search_result
307 https://www.chitai-gorod.ru/catalog/book/761331/?watch_fromlist=search_result
308 https://www.chitai-gorod.ru/catalog/book/754346/?watch_fromlist=search_result
309 https://www.chitai-gorod.ru/catalog/book/754318/?watch_fromlist=search_result
310 https://www.chitai-gorod.ru/catalog/book/754276/?watch_fromlist=search_result
311 https://www.chitai-gorod.ru/catalog/book/754275/?watch_fromlist=search_result
312 https://www.chitai-gorod.ru/catalog/book/754237/?watch_fromlist=search_result
313 https://www.chitai-gorod.ru/catalog/book/752084/?watch_fromlist=search_result
314 https://www.

402 https://www.chitai-gorod.ru/catalog/book/294246/?watch_fromlist=search_result
403 https://www.chitai-gorod.ru/catalog/book/294153/?watch_fromlist=search_result
404 https://www.chitai-gorod.ru/catalog/book/293953/?watch_fromlist=search_result
405 https://www.chitai-gorod.ru/catalog/book/293201/?watch_fromlist=search_result
406 https://www.chitai-gorod.ru/catalog/book/292872/?watch_fromlist=search_result
407 https://www.chitai-gorod.ru/catalog/book/292318/?watch_fromlist=search_result
408 https://www.chitai-gorod.ru/catalog/book/291593/?watch_fromlist=search_result
409 https://www.chitai-gorod.ru/catalog/book/289861/?watch_fromlist=search_result
410 https://www.chitai-gorod.ru/catalog/book/286345/?watch_fromlist=search_result
411 https://www.chitai-gorod.ru/catalog/book/284263/?watch_fromlist=search_result
412 https://www.chitai-gorod.ru/catalog/book/282313/?watch_fromlist=search_result
413 https://www.chitai-gorod.ru/catalog/book/280147/?watch_fromlist=search_result
414 https://www.

,ID карточки,ID товара:,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Обложка,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена:
152,244296,1400705,9785367009781,Фрай М.,NaN,2009,0,"Амфора, ООО",478,"Книга для таких, как я",https://img-gorod.ru/upload/iblock/73b/73b0845...,NaN,NaN,0,ФРАМ,Твердая бумажная,4000.0,20.5 x 13.2 x 3,NaN,NaN
151,245728,1665067,9785942783372,Фрай М.,NaN,2010,0,"Амфора, ООО",606,Волонтеры вечности,https://img-gorod.ru/upload/iblock/bd5/bd5917e...,NaN,NaN,0,NaN,Твердая бумажная,3000.0,20.7 x 13.8 x 3.8,NaN,NaN
441,246061,1668112,9785699110018,Емец Д.,NaN,2003,0,Эксмо-Пресс,384,Таня Гроттер и Золотая Пиявка,https://img-gorod.ru/upload/iblock/236/236a031...,NaN,NaN,0,Таня Гроттер,Твердая бумажная,NaN,20.6 x 13.5 x 2.3,NaN,NaN
150,246246,1801350,5942783535,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Простые волшебные вещи,https://img-gorod.ru/upload/iblock/82d/82d0b11...,NaN,NaN,0,Лабиринты Эхо,Твердая бумажная,NaN,20.8 x 13.7 x 2.3,NaN,NaN
149,246247,1801351,9785367016239,Фрай М.,NaN,2009,0,"Амфора, ООО",367,Темная сторона,https://img-gorod.ru/upload/iblock/a10/a1063f0...,NaN,NaN,0,Лабиринты Ехо,Твердая бумажная,3000.0,20.6 x 13.5 x 2.3,NaN,NaN
148,246533,1804075,9785942783600,Фрай М.,12+,2013,1,"Амфора, ООО",639,Чужак,https://img-gorod.ru/upload/iblock/2ae/2ae746b...,NaN,NaN,5,Лабиринты Ехо,Твердая бумажная,3000.0,20.7 x 13.6 x 4.1,NaN,NaN
147,246749,1806167,9785942783877,Фрай М.,12+,2013,0,"Амфора, ООО",590,Болтливый мертвец,https://img-gorod.ru/upload/iblock/e40/e406f84...,NaN,Стукалина Марина,0,Лабиринты Эхо,Твердая бумажная,2000.0,20.7 x 13.8 x 3.9,NaN,367.0
146,246913,1807965,9785367003390,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Власть несбывшегося,https://img-gorod.ru/upload/iblock/052/052c143...,NaN,NaN,0,NaN,Твердая бумажная,NaN,20.6 x 13.6 x 2.3,NaN,322.0
145,246914,1807966,9785942783884,Фрай М.,12+,2009,0,"Амфора, ООО",383,Наваждения,https://img-gorod.ru/upload/iblock/6c2/6c220c8...,NaN,NaN,0,Лабиринты Ехо,Твердая бумажная,5000.0,20.6 x 13.7 x 2.5,NaN,427.0
144,247198,1810879,9785367004069,Фрай М.,NaN,2003,0,"Амфора, ООО",557,Гнезда Химер Хроники Овётганны,https://img-gorod.ru/upload/iblock/792/7923d42...,NaN,NaN,0,NaN,Твердая бумажная,NaN,20.7 x 13.8 x 3.4,NaN,NaN


In [7]:
df.dtypes

ID карточки         int64
ID товара:          int64
ISBN               object
Автор              object
Возраст            object
Год издания         int64
Голоса              int64
Издательство       object
Кол-во страниц     object
Название           object
Обложка            object
Переводчик         object
Редактор           object
Рейтинг             int64
Серия              object
Тип обложки        object
Тираж             float64
Формат             object
Художник           object
Цена:             float64
dtype: object

In [8]:
with open('hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False, float_format='%.0f')